In [1]:
from shared.simulators import ConstantEffectSimulator, ModeratedEffectSimulator, DataSimulator, EffectSimulatorCollection
from shared.analyzers import AnalyzerCollection, WLSAnalyzer, IPWAnalyzer, SIPWAnalyzer, MetaAnalyzer
from shared.result_processors import Results
from tqdm import trange
import matplotlib.pyplot as plt
import copy

In [2]:
N = 50
T = 200
alpha = 0.05
n_reps = 100
weight_type = "ATE"

# Effect simulators
effect_simulators = EffectSimulatorCollection()
# effect_simulators.add_effect_simulator(ConstantEffectSimulator("No Effect", N, 0., 0., 0.))
# effect_simulators.add_effect_simulator(ConstantEffectSimulator("Shared Effect", N, 1., 1., 0.))
# effect_simulators.add_effect_simulator(ConstantEffectSimulator("Distinct Effects", N, 1., 1., 0.3))
# effect_simulators.add_effect_simulator(ModeratedEffectSimulator("Moderated Effects", N, 1., 1., 0.3, 1., 0.2))
effect_simulators.add_effect_simulator(ModeratedEffectSimulator("Noisy Moderated Effects", N, 1., 1., 0.3, 1., 0.2, epsilon_sd=1.))

# Simulate effects once and fix
for effect_simulator in effect_simulators:
    effect_simulator.sample_params()

# Analyzers
analyzers = AnalyzerCollection()
wls_analyzer = WLSAnalyzer("WLS", alpha)
analyzers.add_analyzer(wls_analyzer)
# analyzers.add_analyzer(MetaAnalyzer(wls_analyzer, "Meta-WLS", alpha))

ipw_model_analyzer = IPWAnalyzer("IPW-Model", alpha, robust=False)
analyzers.add_analyzer(ipw_model_analyzer)
# analyzers.add_analyzer(MetaAnalyzer(ipw_model_analyzer, "Meta-IPW-Model", alpha))
ipw_robust_analyzer = IPWAnalyzer("IPW-Robust", alpha)
analyzers.add_analyzer(ipw_robust_analyzer)
# analyzers.add_analyzer(MetaAnalyzer(ipw_robust_analyzer, "Meta-IPW-Robust", alpha))

sipw_robust_analyzer = SIPWAnalyzer("SIPW-Model", alpha, robust=False)
analyzers.add_analyzer(sipw_robust_analyzer)
# analyzers.add_analyzer(MetaAnalyzer(sipw_robust_analyzer, "Meta-SIPW-Model", alpha))
sipw_robust_analyzer = SIPWAnalyzer("SIPW-Robust", alpha)
analyzers.add_analyzer(sipw_robust_analyzer)
# analyzers.add_analyzer(MetaAnalyzer(sipw_robust_analyzer, "Meta-SIPW-Robust", alpha))

ipw_model_analyzer = IPWAnalyzer("IPW-DR-Model", alpha, dr=True, robust=False)
analyzers.add_analyzer(ipw_model_analyzer)
# analyzers.add_analyzer(MetaAnalyzer(ipw_model_analyzer, "Meta-IPW-DR-Model", alpha))
ipw_dr_analyzer = IPWAnalyzer("IPW-DR-Robust", alpha, dr=True)
analyzers.add_analyzer(ipw_dr_analyzer)
# analyzers.add_analyzer(MetaAnalyzer(ipw_dr_analyzer, "Meta-IPW-DR-Robust", alpha))

sipw_dr_model_analyzer = SIPWAnalyzer("SIPW-DR-Model", alpha, dr=True, robust=False)
analyzers.add_analyzer(sipw_dr_model_analyzer)
# analyzers.add_analyzer(MetaAnalyzer(sipw_dr_model_analyzer, "Meta-SIPW-DR-Model", alpha))
sipw_dr_robust_analyzer = SIPWAnalyzer("SIPW-DR-Robust", alpha, dr=True)
analyzers.add_analyzer(sipw_dr_robust_analyzer)
# analyzers.add_analyzer(MetaAnalyzer(sipw_dr_robust_analyzer, "Meta-SIPW-DR-Robust", alpha))

# Simulation loop
data_simulator = DataSimulator(N, T).simulate_history()
results = Results(data_simulator, effect_simulators, analyzers, n_reps=n_reps)
for rep_idx in trange(n_reps):
    for effect_simulator in effect_simulators:
        data_simulator = data_simulator.simulate_a()
        data_simulator = data_simulator.simulate_effects(effect_simulator)
        for analyzer in analyzers:          
            # Inferences for theta
            theta_inferences = analyzer.get_theta_inferences(data_simulator)
            results = results.add_theta_inferences(theta_inferences, data_simulator.theta.copy(), rep_idx, effect_simulator.i, analyzer.i)
            
            # Inferences for user estimates
            user_inferences = analyzer.get_user_inferences(data_simulator)
            results = results.add_user_inferences(user_inferences, data_simulator.user_effects.copy(), rep_idx, effect_simulator.i, analyzer.i)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 61.47it/s]


In [3]:
digits = 3
results = results.process(digits=digits)
results.user_results_df

WLS  \
Coefficient Metric       Effect Simulator                         
Intercept   Avg Bias     Noisy Moderated Effects  0.460 (0.003)   
            Avg Sq. Bias Noisy Moderated Effects          0.222   
            MSE          Noisy Moderated Effects  0.261 (0.003)   
            Coverage     Noisy Moderated Effects  0.478 (0.007)   
            CI Length    Noisy Moderated Effects  0.891 (0.001)   

                                                      IPW-Model  \
Coefficient Metric       Effect Simulator                         
Intercept   Avg Bias     Noisy Moderated Effects  0.007 (0.005)   
            Avg Sq. Bias Noisy Moderated Effects          0.001   
            MSE          Noisy Moderated Effects  0.092 (0.002)   
            Coverage     Noisy Moderated Effects  0.932 (0.003)   
            CI Length    Noisy Moderated Effects  1.111 (0.001)   

                                                     IPW-Robust  \
Coefficient Metric       Effect Simulator                         
Intercept   Avg Bias     Noisy Moderated Effects  0.007 (0.005)   
            Avg Sq. Bias Noisy Moderated Effects          0.001   
            MSE          Noisy Moderated Effects  0.092 (0.002)   
            Coverage     Noisy Moderated Effects  0.968 (0.002)   
            CI Length    Noisy Moderated Effects  1.299 (0.001)   

                                                     SIPW-Model  \
Coefficient Metric       Effect Simulator                         
Intercept   Avg Bias     Noisy Moderated Effects  0.008 (0.004)   
            Avg Sq. Bias Noisy Moderated Effects          0.000   
            MSE          Noisy Moderated Effects  0.053 (0.001)   
            Coverage     Noisy Moderated Effects  0.965 (0.003)   
            CI Length    Noisy Moderated Effects  0.970 (0.001)   

                                                    SIPW-Robust  \
Coefficient Metric       Effect Simulator                         
Intercept   Avg Bias     Noisy Moderated Effects  0.008 (0.004)   
            Avg Sq. Bias Noisy Moderated Effects          0.000   
            MSE          Noisy Moderated Effects  0.053 (0.001)   
            Coverage     Noisy Moderated Effects  0.962 (0.003)   
            CI Length    Noisy Moderated Effects  0.965 (0.001)   

                                                   IPW-DR-Model  \
Coefficient Metric       Effect Simulator                         
Intercept   Avg Bias     Noisy Moderated Effects  0.007 (0.004)   
            Avg Sq. Bias Noisy Moderated Effects          0.000   
            MSE          Noisy Moderated Effects  0.058 (0.001)   
            Coverage     Noisy Moderated Effects  0.963 (0.003)   
            CI Length    Noisy Moderated Effects  0.992 (0.001)   

                                                  IPW-DR-Robust  \
Coefficient Metric       Effect Simulator                         
Intercept   Avg Bias     Noisy Moderated Effects  0.007 (0.004)   
            Avg Sq. Bias Noisy Moderated Effects          0.000   
            MSE          Noisy Moderated Effects  0.058 (0.001)   
            Coverage     Noisy Moderated Effects  0.962 (0.003)   
            CI Length    Noisy Moderated Effects  1.005 (0.002)   

                                                  SIPW-DR-Model SIPW-DR-Robust  
Coefficient Metric       Effect Simulator                                       
Intercept   Avg Bias     Noisy Moderated Effects  0.011 (0.004)  0.011 (0.004)  
            Avg Sq. Bias Noisy Moderated Effects          0.001          0.001  
            MSE          Noisy Moderated Effects  0.067 (0.002)  0.067 (0.002)  
            Coverage     Noisy Moderated Effects  0.940 (0.003)  0.977 (0.002)  
            CI Length    Noisy Moderated Effects  0.970 (0.001)  1.192 (0.001)

$\tau_i \sim \text{Normal}(\mu, \sigma^2)$

$\hat{\tau}_i \sim \text{Normal}(\tau_i, \omega_i^2)$